In [1]:
!pip install stockstats
!pip install lightgbm
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 4.2 MB/s 
     |████████████████████████████████| 209 kB 44.2 MB/s 
     |████████████████████████████████| 81 kB 9.7 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
     |████████████████████████████████| 147 kB 62.0 MB/s 
     |████████████████████████████████| 112 kB 61.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=51524d83e9e78bbabff686c1438d99ba62f642499b1244f7e74a724c819b5e1c
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os
from enum import IntEnum
import sys
sys.path.append("./utils")

from generate_features import *
from get_targets import *

from stockstats import StockDataFrame

from lightgbm import LGBMClassifier

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

import optuna

In [3]:
best_params = {
    'time_lag': 46,
    'window': 30,
    # 'factor': 14.825366367433316,
    'factor': 3,
    'ewm_periods': 19,
    'ewm_alpha': 0.19576050683568483
}

In [4]:
TRAIN_PATH = "GOOG.csv"
MODEL_PATH = "model.txt"

TEST_RATIO = 0.2
EWM_PERIODS = best_params["ewm_periods"]
EWM_ALPHA = best_params["ewm_alpha"]

TIME_LAG = best_params["time_lag"]
WINDOW = best_params["window"]
FACTOR = best_params["factor"]

In [10]:
train_df = pd.read_csv(TRAIN_PATH)
train_df.reset_index(drop=True, inplace=True)
train_df = indicators_with_rowwise_ewm(train_df)
train_df

,volume_26_ema,volume_42_ema,volume_86_ema,volume_174_ema,volume_363_ema,volume_42_smma,volume_14_mstd,volume_26_mstd,volume_42_mstd,volume_86_mstd,...,supertrend_ub,supertrend_lb,close,high,low,open,volume,boll_dif,boll_ub_close,boll_lb_close
0,0.202800,0.202800,0.202800,0.202800,0.202800,0.202800,NaN,NaN,NaN,NaN,...,841.715000,831.935000,836.020,837.640,836.010,837.060,0.202800,NaN,NaN,NaN
1,0.187223,0.187443,0.187626,0.187714,0.187759,0.187619,0.021213,0.021213,0.021213,0.021213,...,839.102778,832.807222,835.700,836.210,835.700,836.020,0.172800,0.905097,0.612548,0.292548
2,0.165801,0.166538,0.167150,0.167445,0.167595,0.167129,0.037855,0.037855,0.037855,0.037855,...,837.932605,833.097395,835.400,835.700,835.330,835.700,0.127600,1.240215,0.926774,0.313441
3,0.169938,0.170308,0.170631,0.170791,0.170873,0.170619,0.031571,0.031571,0.031571,0.031571,...,837.545047,833.554953,835.700,835.700,835.400,835.660,0.180600,1.012719,0.511360,0.501360
4,0.186325,0.185737,0.185283,0.185075,0.184973,0.185298,0.041446,0.041446,0.041446,0.041446,...,837.492238,833.554953,834.670,835.750,834.670,835.700,0.240600,2.048687,1.852344,0.196344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499858,0.000010,0.000065,0.030533,0.634213,3.122382,0.024575,0.000000,0.000049,0.000098,0.000000,...,2668.498263,2659.901759,2664.827,2668.487,2664.627,2668.016,0.000010,9.866421,5.505761,4.360661
499859,0.000010,0.000062,0.029831,0.626965,3.105226,0.023990,0.000000,0.000048,0.000098,0.000000,...,2668.498263,2659.901759,2665.756,2665.976,2663.656,2664.827,0.000011,9.259379,4.108240,5.151140
499860,0.000010,0.000060,0.029146,0.619800,3.088164,0.023419,0.000000,0.000049,0.000098,0.000000,...,2668.498263,2659.901759,2666.007,2666.436,2665.316,2665.776,0.000006,8.323770,3.207435,5.116335
499861,0.000010,0.000057,0.028476,0.612716,3.071197,0.022861,0.000000,0.000049,0.000098,0.000000,...,2668.498263,2659.901759,2665.037,2666.047,2664.997,2666.027,0.000007,7.875695,3.809398,4.066298


In [11]:
pd.read_csv(TRAIN_PATH)

,Unnamed: 0,Gmt time,Open,High,Low,Close,Volume
0,0,2017-01-26 14:30:00,837.060,837.640,836.010,836.020,0.202800
1,1,2017-01-26 14:31:00,836.020,836.210,835.700,835.700,0.172800
2,2,2017-01-26 14:32:00,835.700,835.700,835.330,835.400,0.127600
3,3,2017-01-26 14:33:00,835.660,835.700,835.400,835.700,0.180600
4,4,2017-01-26 14:34:00,835.700,835.750,834.670,834.670,0.240600
...,...,...,...,...,...,...,...
499858,499858,2022-03-09 18:54:00,2668.016,2668.487,2664.627,2664.827,0.000010
499859,499859,2022-03-09 18:55:00,2664.827,2665.976,2663.656,2665.756,0.000011
499860,499860,2022-03-09 18:56:00,2665.776,2666.436,2665.316,2666.007,0.000006
499861,499861,2022-03-09 18:57:00,2666.027,2666.047,2664.997,2665.037,0.000007


In [12]:
targets = get_abnormal_returns_with_std_target(train_df, TIME_LAG, WINDOW, FACTOR)
train_df = pd.concat([train_df, targets], axis=1)
train_df.dropna(inplace=True)
train_df.reset_index(drop=True, inplace=True)
train_df

,volume_26_ema,volume_42_ema,volume_86_ema,volume_174_ema,volume_363_ema,volume_42_smma,volume_14_mstd,volume_26_mstd,volume_42_mstd,volume_86_mstd,...,supertrend_lb,close,high,low,open,volume,boll_dif,boll_ub_close,boll_lb_close,target
0,0.187223,0.187443,0.187626,0.187714,0.187759,0.187619,0.021213,0.021213,0.021213,0.021213,...,832.807222,835.700,836.210,835.700,836.020,0.172800,0.905097,0.612548,0.292548,0.0
1,0.165801,0.166538,0.167150,0.167445,0.167595,0.167129,0.037855,0.037855,0.037855,0.037855,...,833.097395,835.400,835.700,835.330,835.700,0.127600,1.240215,0.926774,0.313441,0.0
2,0.169938,0.170308,0.170631,0.170791,0.170873,0.170619,0.031571,0.031571,0.031571,0.031571,...,833.554953,835.700,835.700,835.400,835.660,0.180600,1.012719,0.511360,0.501360,0.0
3,0.186325,0.185737,0.185283,0.185075,0.184973,0.185298,0.041446,0.041446,0.041446,0.041446,...,833.554953,834.670,835.750,834.670,835.700,0.240600,2.048687,1.852344,0.196344,0.0
4,0.206229,0.204442,0.203007,0.202332,0.201993,0.203056,0.055393,0.055393,0.055393,0.055393,...,833.554953,834.000,834.870,834.000,834.670,0.285700,3.056421,2.776544,0.279877,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499811,0.000015,0.000493,0.088977,1.076121,4.023047,0.074433,0.000000,0.000049,0.000098,0.000000,...,2660.992819,2664.776,2666.016,2662.846,2666.016,0.000012,21.491216,16.807458,4.683758,0.0
499812,0.000015,0.000471,0.086932,1.063823,4.000942,0.072661,0.000000,0.000049,0.000098,0.000000,...,2660.992819,2662.576,2664.997,2661.506,2664.797,0.000016,20.666459,17.698579,2.967879,0.0
499813,0.000015,0.000450,0.084934,1.051665,3.978959,0.070931,0.000000,0.000049,0.000098,0.000000,...,2660.992819,2666.066,2666.067,2661.686,2662.136,0.000023,19.533938,13.054319,6.479619,0.0
499814,0.000016,0.000430,0.082982,1.039646,3.957097,0.069243,0.000000,0.000049,0.000098,0.000000,...,2660.992819,2666.776,2667.467,2663.097,2665.176,0.000018,17.924059,10.985879,6.938179,0.0


In [13]:
X = train_df.drop(["target"], axis=1).to_numpy()
y = train_df["target"].to_numpy()

X.shape, y.shape

((499816, 74), (499816,))

In [14]:
def get_sample_weights(y: np.ndarray, class_weights: np.ndarray) -> np.ndarray:
    weights = np.zeros(len(y))
    for current_class in np.unique(y):
        weights[y == current_class] = class_weights[int(current_class)]
        
    return weights

In [15]:
test_size = int(X.shape[0] * TEST_RATIO)

X_train = X[:-test_size]
y_train = y[:-test_size]
X_test = X[-test_size:]
y_test = y[-test_size:]

In [16]:
lgbm_classifier = LGBMClassifier(class_weight="balanced")
lgbm_classifier.fit(X_train, y_train)

LGBMClassifier(class_weight='balanced')

In [17]:
probas = lgbm_classifier.predict_proba(X_test)
predictions = lgbm_classifier.predict(X_test)
print(roc_auc_score(y_test, probas[:, 1]))
confusion_matrix(y_test, predictions)

0.6868376791336037


array([[41731, 20343],
       [15541, 22348]])

In [18]:
predictions = lgbm_classifier.predict_proba(X_test)
print(len(predictions))
filtered = list(filter(lambda row: (row[0] >= 0.7 or row[1] >= 0.7), predictions))
print(len(filtered))
filtered = list(filter(lambda row: (row[0] >= 0.8 or row[1] >= 0.8), predictions))
print(len(filtered))

predictions_results = np.array(np.concatenate((np.array(predictions), np.matrix(y_test).T), axis=1))
filtered = list(filter(lambda row: (row[0] >= 0.8 or row[1] >= 0.8), predictions_results))
splited = np.split(filtered, [2, 3], axis=1)
frame = pd.DataFrame(splited[0])
frame['pred'] = frame.apply(lambda row: 0. if row[0] > row[1] else 1., axis=1)
frame
confusion_matrix(splited[1], frame['pred'])

99963
37206
14553


array([[12063,   269],
       [ 1677,   544]])

In [19]:
y_test.shape

(99963,)

In [20]:
lgbm_classifier.booster_.save_model(MODEL_PATH)